In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import Row
from pyspark.sql import types as T

In [ ]:
spark = SparkSession.builder.master("local[*]").appName('ex5_google_reviews').getOrCreate()

In [ ]:
sentiment_arr = [Row(Sentiment='Positive', sentiment_rank=1),
                 Row(Sentiment='Neutral', sentiment_rank=0),
                 Row(Sentiment='Negative', sentiment_rank=-1)]

In [ ]:
google_reviews_df = spark.read.csv('hdfs://course-hdfs:8020/data/raw/google_reviews/', header=True)
sentiments_df = spark.createDataFrame(sentiment_arr)

In [ ]:
joined_df = google_reviews_df.join(F.broadcast(sentiments_df), ['Sentiment'])

In [ ]:
selected_df = joined_df\
    .select(F.col('App').alias('application_name'),
            F.col('Translated_Review').alias('translated_review'),
            F.col('sentiment_rank'),
            F.col('Sentiment_Polarity').cast(T.FloatType()).alias('sentiment_polarity'),
            F.col('Sentiment_Subjectivity').cast(T.FloatType()).alias('sentiment_subjectivity'))

In [ ]:
selected_df.write.parquet('hdfs://course-hdfs:8020/data/source/google_reviews', mode='overwrite')

In [ ]:
spark.stop()